<h2>Simple MNIST convnet - SR Nov21</h2>c.f. https://keras.io/examples/vision/mnist_convnet/

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
model = keras.Sequential(
    [   layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [34]:
batch_size = 128
epochs = 15
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 7s 17ms/step - loss: 0.0455 - accuracy: 0.9853 - val_loss: 0.0349 - val_accuracy: 0.9910
Epoch 2/15
422/422 [==============================] - 7s 16ms/step - loss: 0.0420 - accuracy: 0.9870 - val_loss: 0.0320 - val_accuracy: 0.9922
Epoch 3/15
422/422 [==============================] - 7s 16ms/step - loss: 0.0410 - accuracy: 0.9870 - val_loss: 0.0301 - val_accuracy: 0.9922
Epoch 4/15
422/422 [==============================] - 7s 16ms/step - loss: 0.0385 - accuracy: 0.9875 - val_loss: 0.0277 - val_accuracy: 0.9925
Epoch 5/15
422/422 [==============================] - 7s 16ms/step - loss: 0.0370 - accuracy: 0.9885 - val_loss: 0.0291 - val_accuracy: 0.9923
Epoch 6/15
422/422 [==============================] - 7s 17ms/step - loss: 0.0353 - accuracy: 0.9887 - val_loss: 0.0279 - val_accuracy: 0.9922
Epoch 7/15
422/422 [==============================] - 7s 17ms/step - loss: 0.0328 - accuracy: 0.9893 - val_loss: 0.0275 - val_accuracy: 0.9923

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.029487106949090958
Test accuracy: 0.9886999726295471


OK, now let us play with the network and supply it with our own data...

In [24]:
x_test.shape, y_test.shape

((10000, 28, 28, 1), (10000, 10))

In [31]:
model.predict(x_test[0].reshape(1,28,28,1))

array([[1.6251480e-09, 4.3673240e-10, 2.1971598e-06, 1.3731418e-06,
        4.4243136e-13, 2.4493607e-10, 1.9594806e-16, 9.9999630e-01,
        5.6071374e-09, 1.2497229e-07]], dtype=float32)

In [33]:
test=1-np.fromfile('test-image2.data',dtype=np.ubyte)/255
model.predict(x_test[0].reshape(1,28,28,1))

array([[1.6251480e-09, 4.3673240e-10, 2.1971598e-06, 1.3731418e-06,
        4.4243136e-13, 2.4493607e-10, 1.9594806e-16, 9.9999630e-01,
        5.6071374e-09, 1.2497229e-07]], dtype=float32)